In [1]:
import pandas as pd
import numpy as np

In [2]:
ratings = pd.read_csv('data/BX-Book-Ratings.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251')

In [3]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
User-ID        1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 26.3+ MB


In [4]:
books = pd.read_csv('data/BX-Books.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251')

C:\Users\Efim_Golovin\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271379 entries, 0 to 271378
Data columns (total 8 columns):
ISBN                   271379 non-null object
Book-Title             271379 non-null object
Book-Author            271378 non-null object
Year-Of-Publication    271379 non-null object
Publisher              271377 non-null object
Image-URL-S            271379 non-null object
Image-URL-M            271379 non-null object
Image-URL-L            271376 non-null object
dtypes: object(8)
memory usage: 16.6+ MB


In [6]:
books = books[~books['Year-Of-Publication'].isin(['DK Publishing Inc', 'Gallimard'])]

In [7]:
books['Year-Of-Publication'] = books['Year-Of-Publication'].astype(np.int)

In [8]:
books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271376 entries, 0 to 271378
Data columns (total 8 columns):
ISBN                   271376 non-null object
Book-Title             271376 non-null object
Book-Author            271375 non-null object
Year-Of-Publication    271376 non-null int32
Publisher              271374 non-null object
Image-URL-S            271376 non-null object
Image-URL-M            271376 non-null object
Image-URL-L            271376 non-null object
dtypes: int32(1), object(7)
memory usage: 17.6+ MB


In [9]:
users = pd.read_csv('data/BX-Users.csv', sep=';', header=0, error_bad_lines=False, encoding='Windows-1251')

In [10]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278858 entries, 0 to 278857
Data columns (total 3 columns):
User-ID     278858 non-null int64
Location    278858 non-null object
Age         168096 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.4+ MB


In [ ]:
len(ratings[ratings['Book-Rating'] > 0]['User-ID'].values)

In [ ]:
len(ratings[ratings['Book-Rating'] > 0]['ISBN'].values)

In [ ]:
user_ids_dict = dict(enumerate(users['User-ID']))

In [ ]:
user_ids_dict = dict(zip(user_ids_dict.values(), user_ids_dict.keys()))

In [ ]:
book_isbns_dict = dict(enumerate(books['ISBN']))

In [ ]:
book_isbns_dict = dict(zip(book_isbns_dict.values(), book_isbns_dict.keys()))

In [11]:
arr = np.array([1, 2, 3, 4, 5, 6, 76])

In [19]:
arr[[0, 1, 2, 3]].tolist()

[1, 2, 3, 4]

In [17]:
np.random.randint(len(arr), size=3)

array([6, 4, 0])